# Selenium Import

In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [921 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,552 kB]
Hi

In [2]:
import selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import time

# Crawling

In [48]:
target = input('확인을 원하는 시즌: ')

확인을 원하는 시즌: 2022


In [49]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument("--single-process")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

In [50]:
driver.get('https://www.koreabaseball.com/Schedule/Schedule.aspx')
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="f5efdce065d37b53c9e793752fc00790")>


In [51]:
score = {}
conceded = {}
dif = {}

In [52]:
Select(driver.find_element(By.ID, 'ddlYear')).select_by_value(target)
Select(driver.find_element(By.ID, 'ddlMonth')).select_by_value('12')

season = Select(driver.find_element(By.ID, 'ddlYear')).first_selected_option.text
print(season)

while (True):
  print(Select(driver.find_element(By.ID, 'ddlMonth')).first_selected_option.text)
  gamedata = driver.find_elements(By.CLASS_NAME, 'play')
  for i in range(len(gamedata)):
    spans = gamedata[i].find_elements(By.CSS_SELECTOR, 'span')
    if len(spans) < 5:
      continue
    if spans[0].text in score.keys():
      score[spans[0].text].append(int(spans[1].text))
      conceded[spans[0].text].append(int(spans[3].text))
      dif[spans[0].text].append(int(spans[1].text) - int(spans[3].text))
    else:
      score[spans[0].text] = [int(spans[1].text)]
      conceded[spans[0].text] = [int(spans[3].text)]
      dif[spans[0].text] = [int(spans[1].text) - int(spans[3].text)]
    if spans[4].text in score.keys():
      score[spans[4].text].append(int(spans[3].text))
      conceded[spans[4].text].append(int(spans[1].text))
      dif[spans[4].text].append(int(spans[3].text) - int(spans[1].text))
    else:
      score[spans[4].text] = [int(spans[3].text)]
      conceded[spans[4].text] = [int(spans[1].text)]
      dif[spans[4].text] = [int(spans[3].text) - int(spans[1].text)]

  prevBtn = driver.find_element(By.ID, 'btnPrev')
  prevBtn.click()
  time.sleep(1)
  if Select(driver.find_element(By.ID, 'ddlYear')).first_selected_option.text != season:
    break

2022
12
11
10
09
08
07
06
05
04
03
02
01


# 데이터 분석

## 표준편차 구하기

In [53]:
def calc_statistic(dic):
  avg = {}
  vsum = {}

  for i in dic.keys():
    avg[i] = sum(dic[i]) / len(dic[i])
    v = 0;
    for j in range(len(dic[i])):
      v = v + (dic[i][j] - avg[i]) ** 2
    vsum[i] = (v / len(dic[i])) ** 0.5;
    print(f'{i}\t{avg[i]}\t{vsum[i]}')

  return avg, vsum

In [54]:
print('팀 당 득점 표준편차')
print('팀\t평균득점\t\t표준편차')

temp, score_vsum = calc_statistic(score)

팀 당 득점 표준편차
팀	평균득점		표준편차
NC	4.486111111111111	3.7192930016467862
LG	4.965277777777778	3.4144569977532533
두산	4.430555555555555	3.8254498242722645
삼성	4.604166666666667	3.0623582733645724
SSG	5.0	3.34995854037363
KIA	5.0	3.851406669430448
롯데	4.201388888888889	2.9943925861310405
한화	3.9166666666666665	2.9190466480228316
KT	4.381944444444445	3.24036290786871
키움	4.3125	3.1720759720130003


In [55]:
print('팀 당 실점 표준편차')
print('팀\t평균실점\t\t표준편차')

temp, conceded_vsum = calc_statistic(conceded)

팀 당 실점 표준편차
팀	평균실점		표준편차
NC	4.458333333333333	3.5293904887702943
LG	3.6180555555555554	2.71312787863058
두산	5.006944444444445	3.4630921631176528
삼성	4.826388888888889	3.3836409160496674
SSG	4.319444444444445	3.4533654635284927
KIA	4.715277777777778	2.914673591512732
롯데	4.944444444444445	3.82567673348596
한화	5.270833333333333	3.7661112237957064
KT	3.9027777777777777	2.991468063148846
키움	4.236111111111111	3.306109897495995


In [56]:
print('팀 당 실점 표준편차')
print('팀\t평균실점\t\t표준편차')

temp, dif_vsum = calc_statistic(dif)

ret = {}

for i in dif_vsum.keys():
  ret[i] = -(temp[i] - dif_vsum[i])

팀 당 실점 표준편차
팀	평균실점		표준편차
NC	0.027777777777777776	5.299298229803173
LG	1.3472222222222223	4.2284082972668395
두산	-0.5763888888888888	5.036781188185002
삼성	-0.2222222222222222	4.298650376772735
SSG	0.6805555555555556	4.5926462623805495
KIA	0.2847222222222222	4.905585244342022
롯데	-0.7430555555555556	5.061766445863452
한화	-1.3541666666666667	4.64798156610898
KT	0.4791666666666667	4.718416562906579
키움	0.0763888888888889	4.4317037435942925


## 표준편차 순위

In [57]:
def show_rank(dic):
  ret = dict(sorted(dic.items(), key=lambda item: item[1]))
  rank = 1
  for i in ret.keys():
    print(f'{rank}\t{i}\t{ret[i]}')
    rank += 1

In [58]:
print('팀 당 득점 표준편차 순위')
show_rank(score_vsum)

팀 당 득점 표준편차 순위
1	한화	2.9190466480228316
2	롯데	2.9943925861310405
3	삼성	3.0623582733645724
4	키움	3.1720759720130003
5	KT	3.24036290786871
6	SSG	3.34995854037363
7	LG	3.4144569977532533
8	NC	3.7192930016467862
9	두산	3.8254498242722645
10	KIA	3.851406669430448


In [59]:
print('팀 당 실점 표준편차 순위')
show_rank(conceded_vsum)

팀 당 실점 표준편차 순위
1	LG	2.71312787863058
2	KIA	2.914673591512732
3	KT	2.991468063148846
4	키움	3.306109897495995
5	삼성	3.3836409160496674
6	SSG	3.4533654635284927
7	두산	3.4630921631176528
8	NC	3.5293904887702943
9	한화	3.7661112237957064
10	롯데	3.82567673348596


In [60]:
print('팀 당 득실차 표준편차 순위')
show_rank(dif_vsum)

팀 당 득실차 표준편차 순위
1	LG	4.2284082972668395
2	삼성	4.298650376772735
3	키움	4.4317037435942925
4	SSG	4.5926462623805495
5	한화	4.64798156610898
6	KT	4.718416562906579
7	KIA	4.905585244342022
8	두산	5.036781188185002
9	롯데	5.061766445863452
10	NC	5.299298229803173


In [61]:
print('뭔가 새로운 지표')
show_rank(ret)

뭔가 새로운 지표
1	LG	2.881186075044617
2	SSG	3.912090706824994
3	KT	4.239249896239912
4	키움	4.355314854705403
5	삼성	4.520872598994957
6	KIA	4.620863022119799
7	NC	5.2715204520253955
8	두산	5.613170077073891
9	롯데	5.804822001419008
10	한화	6.002148232775647
